In [ ]:
# ver.1.3 （Youtube公開用)

# colabに標準的にインストールされていないパッケージをインストール
!pip install numpy-quaternion
!pip install opencv-python
!pip install datetime

# GPU, TPU上で稼働させるためのdummy
!pip install tensorflow # CPUおよびGPU(T4)では不要！ TPU(v28)の場合 install必要

# パッケージのインポート
import os
import pandas as x_datas
import time
import numpy as np
import quaternion
import cv2
from scipy.interpolate import CubicSpline
import datetime
import math
import glob
import sys
import subprocess
from matplotlib import pyplot as plt
import json

import tensorflow as tf

# カレントパスの取得
current_path = os.getcwd()
print('colab is running on ', current_path)

# MyDriveにアクセスできるように設定　➡　本プログラムの起動初回時に、いくつかセキュリティチェックがある
from google.colab import drive
content_drive='/content/drive'
drive.mount(content_drive)

# @title 電子手振れ補正＆水平維持　【映像要件】 GoPro11,12,13 / 5.3K 8:7 30FPS 広角 HyperSmooth Off
# @markdown ## 動画mp4ファイルのパスとファイル名の入力

path = "MyDrive/my_project" # @param {type:"string"}
# @markdown - （注） フォルダの区切り文字は、バックスラッシュ「\」や円「¥」ではなく、スラッシュ「/」(割算記号)
filename="GX010822.mp4" # @param {type:"string"}
# @markdown - 映像要件を満たせば、TimeWarp x5 動画（RealSpeed切替を含む）を認識し、水平維持がかけられる
# @markdown - TimeWarp x5 区間：xy平面、広角(1.0)、pitch_offset=0°
# @markdown - RealSpeed 区間：roll軸、リニア(0.0)、pitch_offset=0°

# 'my project'内の 'kalman_def_2.py'ファイルにアクセスできるように設定
append_path=content_drive+'/'+path
#sys.path.append('/content/drive/MyDrive/my_project')
sys.path.append(append_path)
import kalman_def_2

# mp4動画ファイルのメタデータからstreams情報を取得
# local PCに、ffmpegをインストール必要なし
file_name=filename[:-4] # 'GX010785'
mp4_path=content_drive + '/' + path +  '/' + file_name +'.MP4'# '/content/drive/MyDrive/my_project/GX010785.MP4'
try:
  command=['ffprobe','-i',mp4_path,'-hide_banner','-show_streams','-of','json', '-of', 'json']
  result = subprocess.run(command, capture_output=True, text=True, check=True, encoding='utf-8')
except subprocess.CalledProcessError as e:
  print(f"ffprobe error: {e.stderr}")
except json.JSONDecodeError as e:
  print(f"JSON decoding error: {e}")

# 取得したstreams情報をコンソールに表示
video_info = json.loads(result.stdout) # stremas情報を「辞書」データに変換
print('streams情報：　',video_info)

# streams情報を使って、「GoPro 5.3K, 8:7, 30FPS）」のファイルであるかどうか、映像要件を確認
is_GoPro=False
if 'streams' in video_info :
  if len(video_info['streams']) == 4:
    if 'width' in video_info['streams'][0] and 'height' in video_info['streams'][0] and 'r_frame_rate' in video_info['streams'][0]:
      width = video_info['streams'][0]['width']
      height = video_info['streams'][0]['height']
      r_frame_rate = video_info['streams'][0]['r_frame_rate']
      if width == 5312 and height == 4648 and r_frame_rate == '30000/1001':
        if 'codec_tag_string' in video_info['streams'][3]:
          codec_tag_string = video_info['streams'][3]['codec_tag_string']
          if codec_tag_string =='gpmd':
            print(codec_tag_string,'found: binary data is formatted by GoPro') # 要件を満たせば、コンソールに、この表示
            is_GoPro=True
            print('this video is GoPro 5.3K 8:7 30FPS')

print("tensorflow version: ",tf.__version__) # tensorflowのインポート確認

if is_GoPro == False:
  print('this video is not GoPro 5.3K 8:7 30FPS') # 要件を満たさなければ、コンソールに、この表示
  print('要件を満たす動画ファイルを入力してください')



the path where colab is running  /content
Mounted at /content/drive
streams情報：　 {'streams': [{'index': 0, 'codec_name': 'hevc', 'codec_long_name': 'H.265 / HEVC (High Efficiency Video Coding)', 'profile': 'Main 10', 'codec_type': 'video', 'codec_tag_string': 'hvc1', 'codec_tag': '0x31637668', 'width': 5312, 'height': 4648, 'coded_width': 5312, 'coded_height': 4648, 'closed_captions': 0, 'has_b_frames': 0, 'sample_aspect_ratio': '1:1', 'display_aspect_ratio': '8:7', 'pix_fmt': 'yuv420p10le', 'level': 180, 'color_range': 'pc', 'color_space': 'bt709', 'color_transfer': 'bt709', 'color_primaries': 'bt709', 'chroma_location': 'left', 'refs': 1, 'r_frame_rate': '30000/1001', 'avg_frame_rate': '30000/1001', 'time_base': '1/30000', 'start_pts': 0, 'start_time': '0.000000', 'duration_ts': 330330, 'duration': '11.011000', 'bit_rate': '119134917', 'nb_frames': '330', 'disposition': {'default': 1, 'dub': 0, 'original': 0, 'comment': 0, 'lyrics': 0, 'karaoke': 0, 'forced': 0, 'hearing_impaired': 0,

In [ ]:
# 設定条件の入力

# @markdown ## 水平維持の軸選定
horizon_selection = "xy-平面" # @param ["xy-平面", "roll軸", "なし"]
# @markdown ## 四元数の計算
quaternion_selection="GYRO" # @param ["CORI", "GYRO"]
# @markdown ## 同期調整 δ（FPS時間単位）
delta = 0.52 # @param {type:"number"}
# @markdown ## 重力ベクトルの計算
gravity_selection="Kalman" # @param ["GRAV","Kalman"]
# @markdown ---
# @markdown ## pitch軸のオフセット（単位：度）
# @title スライダー フィールド { run: "auto" }
# @markdown - ＋側がカメラ下向き方向
pitch_offset = -13 # @param {type:"slider", min:-30, max:30, step:1}
# @markdown ## 出力映像のリニア(0.0)　⇔　広角(1.0)
## @markdown - 0.0がリニア、1.0が広角最大
# @title スライダー フィールド { run: "auto"
linearity = 0.2 # @param {type:"slider", min:0, max:1, step:0.1}

# 一連の設定条件をコンソールに表示
print('path ', path)
print('filename ', filename)
print('水平維持の軸選定 ', horizon_selection)
print('四元数の計算 ', quaternion_selection)
print('同期調整 δ ', delta)
print('重力ベクトルの計算 ', gravity_selection)
print('pitch軸のオフセット ', pitch_offset)
print('リニア/広角 ', linearity)

path  MyDrive/my_project
filename  GX010822.mp4
水平維持の軸選定  xy-平面
四元数の計算  GYRO
同期調整 δ  0.52
重力ベクトルの計算  Kalman
pitch軸のオフセット  -13
リニア/広角  0.2


In [ ]:
# @markdown # 計算開始 （ 左の▶をクリックし、実行 ）
# @markdown ‐
# @markdown ### ＜処理の流れ（概略）＞
# @markdown ##### ① 音声データ（mp3）の抽出
# @markdown ##### ② バイナリデータ（bin）の抽出
# @markdown ##### ③ 四元数の計算、計算データの出力(csv)
# @markdown ##### ④ 映像変換（広角 ⇒ リニア ⇒ 透視変換 ⇒ 広角）
# @markdown ##### ⑤ 音声データ（mp3）との合体

# tensorflow dummy code
tf.compat.v1.disable_eager_execution() # need to disable eager in TF2.x
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
sess = tf.compat.v1.Session()
# Evaluate the tensor `c`.
print('tensorflow dummy: ',sess.run(c)) # prints 30.0

# 音声データmp3の抽出
mp3_path = content_drive + '/' + path +  '/' + file_name + '.mp3' # '/content/drive/MyDrive/my_project/GX010785.MP3'
command=['ffmpeg','-i',mp4_path,'-vn','-ab','192k',mp3_path]
subprocess.run(command)
print('mp3 audio file exract done')

# binary dataの抽出
bin_path=content_drive + '/' + path +  '/' + file_name +'.bin'    # '/content/drive/MyDrive/my_project/GX010785.bin'
command=['ffmpeg','-y','-i',mp4_path,'-codec','copy','-map','0:3', '-f', 'rawvideo',bin_path]
subprocess.run(command)
print('binary data exract done')

if horizon_selection == "xy-平面":
  horizon_select=0
elif horizon_selection == "roll軸":
  horizon_select=1
else:                     # "なし"
  horizon_select=4
if quaternion_selection == "CORI":
  quaternon_select=1
else:                     # "GYRO"
  quaternion_select=2
if gravity_selection == "GRAV":
  grav_n=0
else:                    # "Kalman"
  grav_n=3
timestamp_shift=delta
pitch_offset=pitch_offset
dist_parameter=linearity
file_name_2 = file_name  # "GX010785"
path_file_2 = content_drive + '/' + path +  '/' + file_name # '/content/drive/MyDrive/my_project/GX010785'
print(file_name_2, path_file_2, horizon_select)

# 電子手振れ補正＆水平維持のための四元数を計算

cori_crop_1,dist_para = kalman_def_2.stab_horizon(file_name_2,path_file_2,horizon_select,quaternion_select,grav_n,timestamp_shift,pitch_offset,dist_parameter)
total_count =len(cori_crop_1) #動画のフレーム総数


# 映像変換（広角→リニア変換、透視変換、リニア→広角変換）

counter = kalman_def_2.stabilization_53_87of18(file_name_2,path_file_2, cori_crop_1, dist_para)

# 映像変換中の進捗および変換速度の確認
for count in counter:
  if count==1:
    start = time.time()
    print(" ")
    print(f"フレーム総数： {total_count} ")
    print("映像変換処理開始")
  if count==50:
    current=time.time()
    time_per_frame="{:.2f}".format((current-start)/count) # 1フレームあたりの処理時間を計算（最初の50フレームの平均）
    print(" ")
    print(f"映像１フレームの処理時間（秒）ー最初の50フレーム平均ー ： {time_per_frame}")
  if count==total_count-1:
    current=time.time()
    time_per_frame="{:.2f}".format((current-start)/count) # 1フレームあたりの処理時間を計算（全フレームの平均）
    print(" ")
    print(f"映像１フレームの処理時間（秒）ー全レーム平均ー： {time_per_frame} ")
    print("映像変換完了")

# 映像変換に要した時間
end = time.time()
SecToConvert = end - start
time_diff = str(datetime.timedelta(seconds=SecToConvert))
print("映像変換に要したトータル時間： ",time_diff[:8])

# 音声のファイルmp3の付加

mp4_path_s = path_file_2 +'_stabilized.mp4'            # '/content/drive/MyDrive/my_project/GX010785'+"_stabilized.mp4"
mp3_path = path_file_2 + '.mp3'                        # '/content/drive/MyDrive/my_project/GX010785'+".mp3"
concat_path = path_file_2+'_stabilized_with_audio.mp4' # '/content/drive/MyDrive/my_project/GX010785'+"_stabilized_with_audio.mp4"
command=['ffmpeg','-i',mp4_path_s,'-i',mp3_path,'-c:v','copy', '-c:a', 'aac','-map','0:v:0','-map','1:a:0',concat_path]
subprocess.run(command)
print("concat done")

# 処理完了
print('all completed')

tf.Tensor(42, shape=(), dtype=int32)
mp3 audio file exract done
binary data exract done
GX010822 /content/drive/MyDrive/my_project/GX010822 0
cv2 video capture succeed
this video is 5312 x 4648, 5.3K
frame count (OpenCV)  330
fps (OpenCV):  29.97
this video is 30fps
duration_sec:  11
GoPro HERO model No. is  12
GYRO(ACCL) data number per sec:  197
this video is  realspeed
ACCL No.:  2174
GYRO No.:  2174
CORI No.:  330
GRAV No.:  330
IORI No.:  330
ACCL,GYRO number(end_timestamp):  2174
CORI,GRAV,IORI number(end_timestamp_1):  330
quaternion calc done
0 
フレーム総数： 330 
映像変換処理開始
49 
映像１フレームの処理時間（秒）ー最初の50フレーム平均ー ： 7.20
328 
映像１フレームの処理時間（秒）ー全レーム平均ー： 7.42 
映像変換完了
映像変換に要したトータル時間：  0:40:41.
concat done
all completed
